This script predicts per hour if the threshold per participant will be met, using different predictive models for an individual, for the whole period.  

In [ ]:
#connect oracle using windows client..
#comment:download correct version:3.5 python+ Oracle 12c  windows 64
#download and pip install cx_Oracle-5.2+oci12c-cp35-none-win_amd64.whl from lfd.uci.edu/~gohlke/pythonlibs/
#cx_oracle 
#commect:Download the right oracle client 12c+ windows 64 and install in C:\instantclient_12_1
#http://www.oracle.com/technetwork/topics/winx64soft-089540.html
#Set path in windows add C:\instantclient_12_1 (use path manager.exe) restart

import cx_Oracle
import pandas as pd
import numpy as np
import time

# classifiers & testing

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
        

import re  
import os 
import pickle
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
import seaborn as sns
import json
import collections
from math import sqrt
from random import randrange
from random import seed


con = cx_Oracle.connect('hanzefitbit/hanzefitbit@127.0.0.1/xe')


#con.close()

In [ ]:
#function to determine if people have reached their goal of the day

def daily_steps_cat_f (steps_value,threshold):
    if (steps_value<threshold):
        #print('smaller then threshold')
        return 0
    if (steps_value>=threshold):
        #print('more then threshold')
        return 1
    

In [ ]:
#function to determine the available models
def get_pickled_models_f (treatment_id, algorithm_name):
    pickle_model=str((treatment_id)+'_'+algorithm_name+'_'+'model.pkl')
    cur.execute("select destination from hft_model_t where name=:pickle_model",pickle_model=pickle_model)
    dest = cur.fetchone()[0]
    globals()['model%s' % treatment_id+algorithm_name] = pickle.load(open(os.path.join(dest,pickle_model), 'rb'))
    cur.execute("select id from hft_model_t where name=:pickle_model",pickle_model=pickle_model)
    globals()['model_id%s' % treatment_id+algorithm_name]=cur.fetchone()[0]

In [ ]:
#function to predict and save the results into the database
def predict_f(treatment_id,algorithm_name):
    cur.execute('select id,treatment_id,year,week,weekday,hour,sum_steps, sum_steps_hour,daily_steps from hft_sum_steps_v where \
             treatment_id=:treatment_id and hour in (7,8,9,10,11,12,13,14,15,16,17,18) \
             and year=2015 and weekday not in (5,6) and (case when research_group=2 and week>15 then 1 when research_group=1 and week>4 then 1 else 0 end = 1)\
             order by year,week,weekday,hour',treatment_id=treatment_id)     
    df= pd.DataFrame(cur.fetchall(),columns=['id','treatment_id','year','week','weekday','hour','sum_steps','sum_steps_hour','daily_steps'])
    
    y_pred = globals()['model%s' % treatment_id+algorithm_name].predict( df.iloc[:, 5:8].values)
    probs_all = globals()['model%s' % treatment_id+algorithm_name].predict_proba(df.iloc[:, 5:8].values)
    #make a dataframe of  globals()['probs_all%s' % treatment_id] to use iloc
    probs_all=pd.DataFrame(probs_all)
    proba_all=probs_all.iloc[:,1]
    
    
    cur.execute( 'select treatment_id,avg(sum_steps_hour) avg_daily_steps from hft_sum_steps_v where treatment_id=:treatment_id \
                  and (case when research_group=2 and week>15 then 1 when research_group=1 and week>4 then 1 else 0 end = 1)\
                  and year=2015 and hour=18 and weekday not in (5,6) group by treatment_id',treatment_id=treatment_id)
    df_threshold= pd.DataFrame(cur.fetchall(),columns=['treatment_id','avg_daily_steps'])

  
    #use numpy vectorize to use function with more arguments and a dataframe determine if the threshold is met

    daily_steps=df['daily_steps']
    threshold=df_threshold['avg_daily_steps']
    df['dailysteps_cat']=np.vectorize(daily_steps_cat_f)(daily_steps, threshold)
    y= df.iloc[:,9].values
    
    #make a dataframe for all predictions
    df_prediction =pd.DataFrame({'id':df.id,'model_id':globals()['model_id%s' % treatment_id+algorithm_name],'probs':proba_all,'y':y,'y_pred': y_pred})
    #insert results into database, dataframe is based on alphabetical order.... so 
    rows = [tuple(x) for x in df_prediction.values]
    cur.prepare("BEGIN INSERT INTO HFT_PREDICTION_T(HFT_DATA_T_ID,HFT_MODEL_T_ID,PROBABILITY,STEPS_CAT,PREDICTION_CAT)VALUES (:1,:2,:3,:4,:5);END;")
    cur.executemany(None,rows)
    con.commit()
    #cur.close()

In [ ]:
# select treatment id's for looping through all models etc...
cur = con.cursor()
cur.execute('select distinct hft_treatment_id from hft_metrics_t where weekday=-1 ')
#dataframe
df_treatment_id=pd.DataFrame(cur.fetchall(),columns=['treatment_id'])

In [ ]:
#Load saved model to test if the model can be retrieved.
for i in df_treatment_id['treatment_id']:
    treatment_id=str(i)
    get_pickled_models_f (treatment_id, 'LR')
    get_pickled_models_f (treatment_id, 'ADA')
    get_pickled_models_f (treatment_id, 'RF')
    get_pickled_models_f (treatment_id, 'KNN')
    get_pickled_models_f (treatment_id, 'NN')
    get_pickled_models_f (treatment_id, 'SGD')
    get_pickled_models_f (treatment_id, 'SVC')
    get_pickled_models_f (treatment_id, 'DT')

In [ ]:
#predict using different algorithms
cur = con.cursor()
sql='DELETE FROM HFT_PREDICTION_T'
con.commit()
cur.execute(sql)
for i in df_treatment_id['treatment_id']:
    treatment_id=int(i)
    predict_f(treatment_id,'LR')
    predict_f(treatment_id,'ADA') 
    predict_f(treatment_id,'RF')
    predict_f(treatment_id,'KNN')
    predict_f(treatment_id,'NN')
    predict_f(treatment_id,'SGD')
    predict_f(treatment_id,'SVC')
    predict_f(treatment_id,'DT')